In [1]:
import time
from datetime import timedelta

import html
import re

import numpy as np
import pandas as pd
import xml.etree.ElementTree as et

import dask
import dask.dataframe as dd
import dask.bag as bd
from dask.distributed import Client
from dask_jobqueue import SLURMCluster

In [2]:
##########################################################
#Step 1. Format 'pmd_rules_results.xml' from Linux terminal#
##########################################################
# -'pmd_rules_results.xml' is not properly formated
# -We used linux format command to format 'pmd_rules_results.xml' to save it in 'pmd_rules_results_fmt.xml'
# $  xmllint --format pmd_rules_results.xml > pmd_rules_results_fmt.xml
##############################################################################
#Step2. Remove some lines in 'test_flaws_rem_sm_tags_fmt' from Linux terminal#
##############################################################################
# remove lines 1 to 2 and the last line of the 'pmd_rules_results_fmt.xml'
# MacOS $  sed -i '' '1,2d;$d' pmd_rules_results_fmt.xml 
# Linux  $  sed -i '1,2d;$d' pmd_rules_results_fmt.xml 
# blocksize breaks it into partitions
#pmd_rules_result_bd = bd.read_text('pmd_rules_results_fmt.xml', blocksize='10MB')
#Use '</violation>\n' as linedelimiter
pmd_rules_result_bd = bd.read_text('pmd_rules_results_fmt.xml', blocksize='5MB')
#pmd_rules_result_bd = bd.read_text('pmd_rules_results_fmt.xml', blocksize=1e8, linedelimiter='</violation>\n')
#pmd_rules_result_bd = bd.read_text('pmd_rules_results_fmt.xml', blocksize=None, linedelimiter='</violation>\n')
#pmd_rules_result_bd = bd.read_text('pmd_rules_results.xml', linedelimiter='</violation>\n')

In [3]:
pmd_rules_result_bd

dask.bag<bag-from-delayed, npartitions=1116>

In [4]:
# look at the first row
pmd_rules_result_bd.take(1)

('  <file name="/scale_wlg_nobackup/filesets/nobackup/uoo03396/SfTI_project_nobackup/SfTI_Projects/StackOverflow_project/my_codesnippet_analysis/codesnippets_java/Code_10000096_9914015_1959_0.java">\n',)

## Start a Dask cluster using SLURM jobs as workers.

There are a couple of things we need to configure here:

disabling the mechanism to write on disk when workers run out of memory,
memory, CPUs, maximum time and number of workers per SLURM job,
dask folders for log files and workers data.
We recommend putting the log folder and workers data folders in your /nesi/nobackup/<project_code> folder, most indicated for temporary files (see NeSI File Systems and Quotas).

All of these options can be set in configuration files, see Dask configuration and Dask jobqueue configuration for more information.

In [5]:
#http://jobqueue.dask.org/en/latest/generated/dask_jobqueue.SLURMCluster.html
dask.config.set(
    {
        "distributed.worker.memory.target": False,  # avoid spilling to disk
        "distributed.worker.memory.spill": False,  # avoid spilling to disk
    }
)
cluster = SLURMCluster(
    cores=10, #cores=24, # we set each job to have 1 Worker, each using 10 cores (threads) and 8 GB of memory
    processes=2,
    memory="8GiB",
    walltime="0-00:30",# walltime="0-00:50",
    log_directory="../dask/logs",  # folder for SLURM logs for each worker
    local_directory="../dask",  # folder for workers data
)

In [6]:
#cluster.adapt(minimum_jobs=20, maximum_jobs=200)
cluster.adapt(minimum_jobs=10, maximum_jobs=200)
client = Client(cluster)
client

Client Scheduler: tcp://192.168.94.167:45211 Dashboard: http://192.168.94.167:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Convert Bags to Dataframes

In [7]:
df = pmd_rules_result_bd.to_dataframe()

In [8]:
# View Stucture
df

,0
npartitions=1116,
,object
,...
...,...
,...
,...


In [9]:
df.partitions[1].compute()

,0
0,Do not start a literal by 0 unless its an octa...
1,</violation>\n
2,"<violation beginline=""2"" endline=""2"" begin..."
3,Do not start a literal by 0 unless its an octa...
4,</violation>\n
...,...
39822,</violation>\n
39823,"<violation beginline=""3"" endline=""6"" begin..."
39824,Do not use non-final non-private static fields\n
39825,</violation>\n


In [10]:
df.columns

RangeIndex(start=0, stop=1, step=1)

In [11]:
#Index will not be particularly meaningful. Use reindex afterwards if necessary.
df.index

Dask Index Structure:
npartitions=1116
    int64
      ...
    ...  
      ...
      ...
dtype: int64
Dask Name: to_dataframe, 4464 tasks

In [12]:
df.dtypes

0    object
dtype: object

In [13]:
df.npartitions

1116

In [14]:
df.shape[0].compute()

57390191

distributed.utils - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/utils.py", line 668, in log_errors
    yield
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 1112, in update
    cpu = [int(ws.metrics["cpu"]) for ws in workers]
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5-gimkl-2020a/lib/python3.9/site-packages/distributed/dashboard/components/scheduler.py", line 1112, in <listcomp>
    cpu = [int(ws.metrics["cpu"]) for ws in workers]
TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'
distributed.utils - ERROR - int() argument must be a string, a bytes-like object or a number, not 'NoneType'
Traceback (most recent call last):
  File "/opt/nesi/CS400_centos7_bdw/Python/3.9.5